In [34]:
import pandas as pd

pd.set_option("display.max_rows", 800)
pd.set_option("display.float_format", "{:.5f}".format)

In [2]:
import os
import time
import glob
import pickle

import pandas as pd
import xarray as xr
import numpy as np


def _convert_forecast_to_historical_units(forecast_df: pd.DataFrame):
    """Converts forecast data units to match historical data units."""
    forecast_df[["tas", "tasmax", "tasmin"]] -= 273.15  # Convert Kelvin to Celsius
    forecast_df[["rsds"]] *= 24  # Convert daily mean to total daily radiation
    return forecast_df


def _extract_point_data(
    dataset: xr.Dataset, lat: float, lon: float, variables: list[str]
):
    """Extracts time series data for specified variables at a given lat/lon."""
    df = dataset[variables].sel(lat=lat, lon=lon, method="nearest").to_dataframe()
    # Optimize datetime conversion by performing it once
    df["iso-date"] = pd.to_datetime(df.index).date
    return df.set_index("iso-date")[variables]


def _combine_dataframes(historical_df: pd.DataFrame, forecast_df: pd.DataFrame):
    """Combines historical and forecast dataframes, removing overlaps."""
    # Find last valid index in historical data more efficiently
    last_valid_historical = historical_df.last_valid_index()
    historical_df = historical_df.loc[:last_valid_historical]

    combined_df = pd.concat([historical_df, forecast_df], axis=0)
    return combined_df[~combined_df.index.duplicated(keep="first")]


def _load_netcdf_data(folder: str, file_pattern: str, start_year: int, end_year: int):
    """Loads NetCDF files within the year range into an xarray dataset."""
    file_paths = []
    for year in range(start_year, end_year + 1):
        file_paths.extend(
            glob.glob(os.path.join(folder, file_pattern.format(year=year)))
        )

    if not file_paths:
        print(f"No matching files found for years {start_year} to {end_year}.")
        return None
    return xr.open_mfdataset(file_paths, parallel=False).load()  # Disable parallel


def extract_variables_over_years(
    netcdf_folder: str,
    variables: list[str],
    file_name_pattern: str,
    start_year: int,
    end_year: int,
    mask_path: str,
    max_points: int = None,  # Limit the number of points processed for testing
):
    """Extracts and combines historical and forecast data for valid grid points."""
    mask = np.load(mask_path)

    historical_ds = _load_netcdf_data(
        netcdf_folder, file_name_pattern, start_year, end_year
    )
    forecast_ds = _load_netcdf_data(
        "netcdf_files/forecasts_2024_05/r1i1p1", "combined.nc", 2024, 2024
    )

    if not historical_ds or not forecast_ds:
        return {}

    lats, lons = historical_ds["lat"].values, historical_ds["lon"].values
    lon_grid, lat_grid = np.meshgrid(lons, lats)
    valid_lat_indices, valid_lon_indices = np.where(mask)

    df_dict = {}
    start_time = time.time()
    for count, (lat_idx, lon_idx) in enumerate(
        zip(valid_lat_indices, valid_lon_indices)
    ):
        lat, lon = lat_grid[lat_idx, lon_idx], lon_grid[lat_idx, lon_idx]

        # Extract data once for both historical and forecast
        historical_df = _extract_point_data(historical_ds, lat, lon, variables)
        forecast_df = _extract_point_data(forecast_ds, lat, lon, variables)
        forecast_df = _convert_forecast_to_historical_units(forecast_df.copy())

        df_dict[f"{lat},{lon}"] = _combine_dataframes(historical_df, forecast_df)

        if max_points and count >= max_points - 1:
            break

    historical_ds.close()
    forecast_ds.close()
    print(f"Processed {count + 1} points in {time.time() - start_time:.2f} seconds.")
    return df_dict

In [5]:
df_dict = extract_variables_over_years(
    netcdf_folder="netcdf_files/combined",
    variables=["hurs", "pr", "rsds", "sfcWind", "tas", "tasmax", "tasmin"],
    file_name_pattern="zalf_combined_amber_{year}_v1-0_uncompressed.nc",
    start_year=2023,
    end_year=2024,
    mask_path="data_availability_mask_h_and_fc.npy",
    max_points=1000,
)
print(len(df_dict))

Processed 1000 points in 6.75 seconds.
1000


In [25]:
import pandas as pd


def save_df_to_csv_monica_conform(df, csv_file_path):
    df = df.rename(
        columns={
            "tas": "tavg",
            "tasmin": "tmin",
            "tasmax": "tmax",
            "sfcWind": "wind",
            "rsds": "globrad",
            "pr": "precip",
            "hurs": "relhumid",
        }
    )

    # Reorder and select columns
    df = df[["tavg", "tmin", "tmax", "wind", "globrad", "precip", "relhumid"]]

    # Convert globrad (rsds) from W/m² to J/cm²
    df["globrad"] = df["globrad"] * 3600 / 10000  # 3600 sec/hour, 1m² = 10000cm²

    df[["tavg", "tmin", "tmax", "wind", "globrad", "precip", "relhumid"]] = df[
        ["tavg", "tmin", "tmax", "wind", "globrad", "precip", "relhumid"]
    ].round(1)

    # Create the header with units AFTER renaming the dataframe
    header = "iso-date;tavg;tmin;tmax;wind;globrad;precip;relhumid\n;C_deg;C_deg;C_deg;m/s;J cm-2;mm;%\n"
    # Open the file in write mode and write the header and data
    with open(csv_file_path, "w") as f:
        f.write(header)
        df.to_csv(f, sep=";", index=True, header=False, date_format="%Y-%m-%d")


df = df_dict["54.80027617931778,9.647312950670669"]
save_df_to_csv_monica_conform(
    df=df, csv_file_path="54.80027617931778,9.647312950670669.csv"
)

In [10]:
df.head(20)

,hurs,pr,rsds,sfcWind,tas,tasmax,tasmin
iso-date,,,,,,,
2023-01-01,84.300003,3.100000,337.100006,9.6,10.500000,12.800000,8.7
2023-01-02,95.000000,4.500000,242.500000,5.8,7.900000,10.300000,5.2
2023-01-03,82.599998,3.400000,724.500000,5.4,4.800000,6.900000,2.5
2023-01-04,93.700005,22.100000,237.099976,11.2,8.300000,10.200000,5.8
2023-01-05,84.900002,0.700000,676.700012,8.1,5.100000,8.000000,2.1
2023-01-06,91.300003,10.700000,367.900024,7.4,7.200000,9.400001,4.4
2023-01-07,89.900002,1.300000,279.600006,5.5,8.400001,10.300000,5.9
2023-01-08,87.400002,0.700000,358.699982,5.0,7.400000,8.900001,4.7
2023-01-09,92.599998,3.500000,325.700012,4.5,5.500000,6.500000,3.8


In [52]:
df = (
    xr.load_dataset("netcdf_files/forecasts_2024_05/r1i1p1/combined.nc")[
        ["hurs", "pr", "rsds", "sfcWind", "tas", "tasmax", "tasmin"]
    ]
    .sel(lat="54.80027617931778", lon="9.647312950670669", method="nearest")
    .to_dataframe()
)
df = _convert_forecast_to_historical_units(df)
start_date = pd.to_datetime("2024-05-28").date()
end_date = pd.to_datetime("2024-06-10").date()
display(df[start_date:end_date])

,hurs,pr,rsds,sfcWind,tas,tasmax,tasmin,lon,lat,height
time,,,,,,,,,,
2024-05-28 12:00:00,65.16186,0.00000,7924.99707,2.93093,13.23093,19.46906,6.34537,9.61250,54.82500,2.00000
2024-05-29 12:00:00,73.58043,0.00005,5227.64258,8.50000,12.50720,15.70001,9.36905,9.61250,54.82500,2.00000
2024-05-30 12:00:00,67.01443,0.00000,4486.24316,5.13814,10.66907,13.93094,6.56186,9.61250,54.82500,2.00000
2024-05-31 12:00:00,66.53093,0.00000,6746.67383,4.06186,11.03094,16.93094,4.66907,9.61250,54.82500,2.00000
2024-06-01 12:00:00,73.78557,0.00005,5195.06055,6.70928,10.89999,16.37628,5.56186,9.61250,54.82500,2.00000
2024-06-02 12:00:00,72.30721,0.00000,6345.67383,8.33814,12.06906,15.19278,9.33813,9.61250,54.82500,2.00000
2024-06-03 12:00:00,75.44536,0.00000,5422.09375,4.20721,12.76907,16.96906,7.86185,9.61250,54.82500,2.00000
2024-06-04 12:00:00,79.15464,0.00012,3034.36646,5.40000,15.13815,17.46906,12.03815,9.61250,54.82500,2.00000
2024-06-05 12:00:00,73.26907,0.00000,7129.90332,2.10000,16.79999,21.76187,10.49280,9.61250,54.82500,2.00000


In [50]:
df = (
    xr.load_dataset(
        "netcdf_files/combined/zalf_combined_amber_2024_v1-0_uncompressed.nc"
    )[["hurs", "pr", "rsds", "sfcWind", "tas", "tasmax", "tasmin"]]
    .sel(lat="54.80027617931778", lon="9.647312950670669", method="nearest")
    .to_dataframe()
)

display(df[start_date:end_date])

,hurs,pr,rsds,sfcWind,tas,tasmax,tasmin,lon,lat,lons,lats,latitude_longitude
time,,,,,,,,,,,,
2024-05-28,81.20000,20.60000,3241.40015,3.50000,13.60000,16.60000,11.00000,9.64731,54.80028,9.64731,54.80028,0
2024-05-29,87.80000,8.00000,2542.50024,3.90000,13.90000,17.20000,12.10000,9.64731,54.80028,9.64731,54.80028,0
2024-05-30,80.60000,0.20000,4357.69971,3.70000,14.80000,19.50000,11.30000,9.64731,54.80028,9.64731,54.80028,0
2024-05-31,82.30000,0.00000,3948.59985,3.60000,15.40000,20.10000,11.20000,9.64731,54.80028,9.64731,54.80028,0
2024-06-01,78.40000,0.00000,5290.00049,4.40000,16.70000,21.80000,12.40000,9.64731,54.80028,9.64731,54.80028,0
2024-06-02,75.90000,0.00000,5721.29980,5.30000,15.90000,20.40000,12.30000,9.64731,54.80028,9.64731,54.80028,0
2024-06-03,74.90000,0.00000,6102.99951,5.30000,16.30000,21.50000,11.90000,9.64731,54.80028,9.64731,54.80028,0
2024-06-04,73.00000,0.00000,6045.90039,5.10000,16.00000,21.00000,12.10000,9.64731,54.80028,9.64731,54.80028,0
2024-06-05,75.50000,0.00000,5558.50049,5.30000,14.80000,19.10000,11.60000,9.64731,54.80028,9.64731,54.80028,0


In [51]:
df = df_dict["54.80027617931778,9.647312950670669"]

display(df[start_date:end_date])

,hurs,pr,rsds,sfcWind,tas,tasmax,tasmin
iso-date,,,,,,,
2024-05-28,81.20000,20.60000,3241.40015,3.50000,13.60000,16.60000,11.00000
2024-05-29,87.80000,8.00000,2542.50024,3.90000,13.90000,17.20000,12.10000
2024-05-30,80.60000,0.20000,4357.69971,3.70000,14.80000,19.50000,11.30000
2024-05-31,82.30000,0.00000,3948.59985,3.60000,15.40000,20.10000,11.20000
2024-06-01,78.40000,0.00000,5290.00049,4.40000,16.70000,21.80000,12.40000
2024-06-02,75.90000,0.00000,5721.29980,5.30000,15.90000,20.40000,12.30000
2024-06-03,74.90000,0.00000,6102.99951,5.30000,16.30000,21.50000,11.90000
2024-06-04,73.00000,0.00000,6045.90039,5.10000,16.00000,21.00000,12.10000
2024-06-05,75.50000,0.00000,5558.50049,5.30000,14.80000,19.10000,11.60000


In [1]:
import time
import pickle

t0 = time.time()
with open("5000_points_extracted.pkl", "rb") as f:
    df_dict_loaded = dict(pickle.load(f))
    df = df_dict_loaded["54.80027617931778,9.647312950670669"]
    display(df)
t1 = time.time()
print(t1 - t0)

,hurs,pr,rsds,sfcWind,tas,tasmax,tasmin
iso-date,,,,,,,
2023-01-01,84.300003,3.100000,337.100006,9.600000,10.500000,12.800000,8.700000
2023-01-02,95.000000,4.500000,242.500000,5.800000,7.900000,10.300000,5.200000
2023-01-03,82.599998,3.400000,724.500000,5.400000,4.800000,6.900000,2.500000
2023-01-04,93.700005,22.100000,237.099976,11.200000,8.300000,10.200000,5.800000
2023-01-05,84.900002,0.700000,676.700012,8.100000,5.100000,8.000000,2.100000
...,...,...,...,...,...,...,...
2024-10-27,94.676292,0.000000,896.561096,3.092785,9.500000,11.861847,5.576294
2024-10-28,91.909286,0.000058,805.484497,5.785571,12.169067,13.338135,10.630951
2024-10-29,87.059792,0.000006,716.387939,5.769072,11.069061,12.438141,9.200012


2.3002564907073975


In [3]:
import numpy as np

with open("data_availability_mask_h_and_fc.pkl", "rb") as f:
    mask = np.array(pickle.load(f))
    np.save("data_availability_mask_h_and_fc.npy", mask)

In [15]:
ds = xr.load_dataset(
    "netcdf_files/forecasts/20240501/r10i1p1/merged_20240501_r10i1p1.nc"
)[["hurs", "pr", "rsds", "sfcWind", "tas", "tasmax", "tasmin"]].sel(
    lat="54.80027617931778", lon="9.647312950670669", method="nearest"
)
print(ds)

<xarray.Dataset> Size: 7kB
Dimensions:  (time: 184)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2024-05-01T12:00:00 ... 2024-10-31T12:...
    lon      float64 8B 9.613
    lat      float64 8B 54.83
    height   float32 4B 2.0
Data variables:
    hurs     (time) float32 736B 75.89 76.97 89.0 90.09 ... 95.59 82.16 88.2
    pr       (time) float32 736B 7.382e-05 0.0 1.157e-05 ... 0.0 0.0 5.465e-05
    rsds     (time) float32 736B 290.6 244.1 220.8 106.2 ... 51.07 70.71 17.75
    sfcWind  (time) float32 736B 3.362 4.593 4.593 3.9 ... 3.531 3.031 7.324
    tas      (time) float32 736B 286.1 285.0 284.4 284.0 ... 282.9 281.0 285.0
    tasmax   (time) float32 736B 291.9 289.5 288.8 289.1 ... 284.7 285.9 285.9
    tasmin   (time) float32 736B 282.0 280.1 281.3 279.7 ... 282.2 278.8 283.3
Attributes: (12/42)
    CDI:                            Climate Data Interface version 2.3.0 (htt...
    Conventions:                    CF-1.4
    source:                         DWD-EPISODES2022
    

In [7]:
xr.load_dataset("netcdf_files/combined/zalf_combined_amber_2024_v1-0_uncompressed.nc")[
    ["hurs", "pr", "rsds", "sfcWind", "tas", "tasmax", "tasmin"]
]["rsds"].sel()

<xarray.DataArray 'rsds' (time: 366, lat: 866, lon: 654)> Size: 829MB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
...
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]],

       [[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * lon                 (lon) float64 5kB 5.873 5.887 5.901 ... 15.02 15.03
  * lat                 (lat) float64 7kB 55.05 55.04 55.03 ... 47.28 47.27
  * time                (time) datetime64[ns] 3kB 2024-01-01 ... 2024-12-31
    lons                (lat, lon) float64 5MB 5.873 5.887 5.901 ... 15.02 15.03
    lats                (lat, lon) float64 5MB 55.05 55.05 55.05 ... 47.27 47.27
    latitude_longitude  int64 8B 0
Attributes: (12/14)
    standard_name:   surface_downwelling_shortwave_flux_in_air
    long_name:       Surface Downwelling Shortwave Radiation
    units:           W m-2
    grid_mapping:    latitude_longitude
    esri_pe_string:  GEOGCS["GCS_unknown",DATUM["D_unknown",SPHEROID["WGS_198...
    institution:     Deutscher Wetterdienst
    ...              ...
    conventions:     CF-1.8
    conventionsURL:  http://cfconventions.org/Data/cf-conventions/cf-conventi...
    description:     The interpolation method is a regional multiple linear r...
    data_version:    v1-0
    creation_date:   2024-05-28 07:46:56
    history:         2023-04-01: Begin of data delivery\n

In [9]:
historical_ds = xr.load_dataset(
    "netcdf_files/amber/2024/zalf_merged_amber_2024_v1-0.nc"
)[["hurs", "pr", "rsds", "sfcWind", "tas", "tasmax", "tasmin"]]
lats, lons = historical_ds["lat"].values, historical_ds["lon"].values
lon_grid, lat_grid = np.meshgrid(lons, lats)

In [10]:
print(lon_grid)

[[ 5.87326547  5.88729539  5.90132531 ... 15.00674097 15.02077088
  15.0348008 ]
 [ 5.87326547  5.88729539  5.90132531 ... 15.00674097 15.02077088
  15.0348008 ]
 [ 5.87326547  5.88729539  5.90132531 ... 15.00674097 15.02077088
  15.0348008 ]
 ...
 [ 5.87326547  5.88729539  5.90132531 ... 15.00674097 15.02077088
  15.0348008 ]
 [ 5.87326547  5.88729539  5.90132531 ... 15.00674097 15.02077088
  15.0348008 ]
 [ 5.87326547  5.88729539  5.90132531 ... 15.00674097 15.02077088
  15.0348008 ]]


100 files pro niederschlag 2 ordner

100 zufaellige punkte keine dublikate in den niedrigaufgeloesten dateien

bezeichnung mit der 1km*1km aufloesung

Punkte nur aus der geotif maske nehmen(fein)

Einheiten pruefen. 



In [109]:
import xarray as xr
import numpy as np

# Load your NetCDF dataset
ds = xr.open_mfdataset(
    paths=[
        "Marlene_netcdf/prAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MQDM-v1-DWD-PCA-v1_day_mm.nc",
        "Marlene_netcdf/rsdsAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MLC-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/sfcWindAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MQDM-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/hursAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MMBC-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/tasmaxAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MMBC-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/tasminAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MMBC-v1-DWD-PCA-v1_day.nc",
    ],
    parallel=True,
)

In [8]:
import xarray as xr
import numpy as np

# Load your NetCDF dataset
ds = xr.open_mfdataset(
    paths=[
        "Marlene_netcdf/prAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MQDM-v1-DWD-PCA-v1_day_mm.nc",
        "Marlene_netcdf/rsdsAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MLC-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/sfcWindAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MQDM-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/hursAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MMBC-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/tasmaxAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MMBC-v1-DWD-PCA-v1_day.nc",
        "Marlene_netcdf/tasminAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MMBC-v1-DWD-PCA-v1_day.nc",
    ],
    parallel=True,
)

encoding = {var: {"zlib": False} for var in ds.data_vars}
ds.to_netcdf('Marlene_netcdf/merged_pr_normal.nc', encoding=encoding)

In [1]:
import xarray as xr
import numpy as np
ds = xr.open_dataset('Marlene_netcdf/merged_pr_normal.nc')

encoding = {var: {"zlib": False,"complevel": 5, "zstd": True} for var in ds.data_vars}
ds.to_netcdf('Marlene_netcdf/merged_pr_normal_zstd.nc', encoding=encoding)

In [9]:
encoding = {var: {"zlib": False} for var in ds.data_vars}
ds.to_netcdf('Marlene_netcdf/merged_pr_normal.nc', encoding=encoding)

In [112]:
def find_nearest_indices(lat_array, lon_array, target_lat, target_lon):
    """
    Finds the indices of the grid points in a 2D array that are closest to the
    target latitude and longitude.

    Args:
        lat_array: 2D array of latitude values.
        lon_array: 2D array of longitude values.
        target_lat: Target latitude.
        target_lon: Target longitude.

    Returns:
        Tuple: (latitude_index, longitude_index) of the nearest grid point.
    """

    # Calculate the distance using broadcasting (Haversine formula for accuracy)
    R = 6371  # Earth's radius in kilometers
    lat1 = np.radians(target_lat)
    lon1 = np.radians(target_lon)
    lat2 = np.radians(lat_array)
    lon2 = np.radians(lon_array)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    distance = R * c

    # Find the indices of the minimum distance
    min_index = np.unravel_index(np.argmin(distance), distance.shape)
    lat_index, lon_index = min_index

    return lat_index, lon_index


# --- Example Usage (assuming your data is in the 'ds' variable) ---

target_longitude = 9.647312950670669
target_latitude = 54.80027617931778

lat_index, lon_index = find_nearest_indices(
    ds["lat"].values, ds["lon"].values, target_latitude, target_longitude
)

print(f"Nearest indices: ({lat_index}, {lon_index})")

# Get the value from your dataset
value = ds[
    [
        "tasmaxAdjustInterp",
        "tasminAdjustInterp",
        "prAdjustInterp",
        "rsdsAdjustInterp",
        "hursAdjustInterp",
        "sfcWindAdjustInterp",
    ]
].isel(x=lon_index, y=lat_index)
dataframe = value.to_dataframe()

Nearest indices: (200, 95)


In [113]:
dataframe.to_csv(f"marlene_{lat_index}_{lon_index}.csv")

In [114]:
display(dataframe)

,tasmaxAdjustInterp,tasminAdjustInterp,rsdsAdjustInterp,hursAdjustInterp,sfcWindAdjustInterp,lon,lat,x,y
time,,,,,,,,,
1951-01-01 12:00:00,274.44998,272.04999,12.00000,98.30000,5.60000,9.63848,54.81112,3977500.00000,3102500.00000
1951-01-02 12:00:00,272.85001,270.54999,13.20000,91.20000,5.30000,9.63848,54.81112,3977500.00000,3102500.00000
1951-01-03 12:00:00,271.64999,269.64999,13.40000,89.10000,4.60000,9.63848,54.81112,3977500.00000,3102500.00000
1951-01-04 12:00:00,276.44998,270.85001,8.60000,97.20000,3.00000,9.63848,54.81112,3977500.00000,3102500.00000
1951-01-05 12:00:00,276.44998,273.44998,11.10000,89.30000,3.00000,9.63848,54.81112,3977500.00000,3102500.00000
...,...,...,...,...,...,...,...,...,...
2100-12-27 12:00:00,287.14999,284.75000,3.30000,97.20000,8.20000,9.63848,54.81112,3977500.00000,3102500.00000
2100-12-28 12:00:00,287.94998,284.75000,2.50000,97.10000,7.60000,9.63848,54.81112,3977500.00000,3102500.00000
2100-12-29 12:00:00,287.64999,279.25000,4.60000,91.10000,7.20000,9.63848,54.81112,3977500.00000,3102500.00000


In [5]:
# Load your NetCDF dataset
import xarray as xr

ds = xr.open_dataset(
    filename_or_obj="Marlene_netcdf/prAdjustInterp_HYR-5_MIROC-MIROC5_historical_and_rcp85_r1i1p1_CLMcom-CCLM4-8-17_v1-DWD-MQDM-v1-DWD-PCA-v1_day_mm.nc"
)
ds.load()

<xarray.Dataset> Size: 12GB
Dimensions:         (time: 54750, y: 220, x: 240, x_2: 2)
Coordinates:
  * time            (time) object 438kB 1951-01-01 12:00:00 ... 2100-12-31 12...
    lon             (y, x) float64 422kB 3.451 3.517 3.582 ... 21.19 21.27 21.35
    lat             (y, x) float64 422kB 45.31 45.31 45.32 ... 55.21 55.2 55.19
  * x               (x) float64 2kB 3.502e+06 3.508e+06 ... 4.692e+06 4.698e+06
  * y               (y) float64 2kB 2.102e+06 2.108e+06 ... 3.192e+06 3.198e+06
Dimensions without coordinates: x_2
Data variables:
    crs             int32 4B 1
    prAdjustInterp  (time, y, x) float32 12GB nan nan nan nan ... nan nan nan
    time_bnds       (time, x_2) datetime64[ns] 876kB 1951-01-01 ... 2100-11-25
Attributes: (12/50)
    CDI:                            Climate Data Interface version 2.2.3 (htt...
    Conventions:                    CF-1.4
    source:                         CORDEX - RCM projection run, surface obse...
    institution:                    Department Climate and Environment Consul...
    title:                          bias-adjusted and interpolated RCM (PR)
    project_id:                     BMDV-ExpN-SP101
    ...                             ...
    input_institution:              Climate Limited-area Modelling Community ...
    input_institute_id:             CLMcom
    experiment_id:                  historical
    driving_experiment:             historical
    experiment:                     historical
    CDO:                            Climate Data Operators version 2.2.1 (htt...

In [6]:
value = ds[
    [
        "prAdjustInterp"
    ]
].isel(x=200, y=95)
dataframe = value.to_dataframe()


In [7]:
display(dataframe)

,prAdjustInterp,lon,lat,x,y
time,,,,,
1951-01-01 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
1951-01-02 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
1951-01-03 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
1951-01-04 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
1951-01-05 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
...,...,...,...,...,...
2100-12-27 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
2100-12-28 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
2100-12-29 12:00:00,NaN,17.223612,49.700037,4502500.0,2577500.0
